# Convert SSD MobileNet from UFF (.uff) to TensorRT engine file (.engine)

NVIDIA has a large selection of pre-trained SSD models in its [Hello AI World guide](https://github.com/dusty-nv/jetson-inference/releases/tag/model-mirror-190618).

This tutorial shows how to convert them to TensorRT engine file.

**NOTE**: NVIDIA has deprecated the Caffe Parser and UFF Parser in TensorRT 7.0. It is expected that converting to ONNX will be required in the future, but for now we will convert from UFF.  
https://docs.nvidia.com/deeplearning/tensorrt/release-notes/tensorrt-7.html

Reference: https://github.com/jkjung-avt/tensorrt_demos

In [ ]:
import numpy as np
import uff
import tensorrt as trt
import graphsurgeon as gs

In [ ]:
# initialize
if trt.__version__[0] < '7':
    try:
        from jnmouse.ssd_tensorrt import load_flattenconcat_plugin
        load_flattenconcat_plugin()
    except:
        import ctypes
        import os
        from pathlib import Path
        import subprocess
        import sys
        command1 = ["cp", "-r", "/usr/src/tensorrt/samples/python/uff_ssd/", "."], Path().resolve() 
        command2 = ["mkdir", "-p", "uff_ssd/build"], Path().resolve()
        command3 = ["cmake", ".."], "{}/uff_ssd/build".format(Path().resolve())
        command4 = ["make"], "{}/uff_ssd/build".format(Path().resolve())
        command5 = ["chmod", "-x", "libflattenconcat.so"], "{}/uff_ssd/build".format(Path().resolve())
        command6 = ["cp", "libflattenconcat.so", "../../libflattenconcat.so.{}".format(trt.__version__[0])], "{}/uff_ssd/build".format(Path().resolve())
        for commands in (command1, command2, command3, command4, command5, command6):
            command, cwd = commands
            print(command)
            res = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=cwd)
            print(res.stdout.decode("utf8"))
            if res.stderr: print(res.stderr.decode("utf8"))

        LIB_FILE = os.path.abspath(os.path.join(Path().resolve(), 'libflattenconcat.so.{}'.format(trt.__version__[0])))
        ctypes.CDLL(LIB_FILE)

TRT_LOGGER = trt.Logger(trt.Logger.INFO)

# load plugins
trt.init_libnvinfer_plugins(TRT_LOGGER, '')

## Download the pre-trained model and configure parameters

Download the pre-trained ssd_mobilenet_v2_coco model. The URL is written in the following script.  
https://github.com/dusty-nv/jetson-inference/blob/master/tools/download-models.sh

In [ ]:
# SSD MobileNetV2 COCO
!wget https://nvidia.box.com/shared/static/jcdewxep8vamzm71zajcovza938lygre.gz -O SSD-Mobilenet-v2.tar.gz
!tar zxvf SSD-Mobilenet-v2.tar.gz

In [ ]:
# SSD MobileNetV2 COCO
uff_model_path = "SSD-Mobilenet-v2/ssd_mobilenet_v2_coco.uff"
trt_engine_path = "ssd_mobilenet_v2_coco_trt{}_t210.engine".format(trt.__version__[0])

print(trt_engine_path)

## Compile the model into TensorRT .engine file

In [ ]:
# compile the model into TensorRT engine
print("build .engine file")
with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.UffParser() as parser:

    builder.max_workspace_size = 1 << 28
    builder.max_batch_size = 1
    builder.fp16_mode = True

    parser.register_input('Input', (3, 300, 300))
    parser.register_output('MarkOutput_0')

    print("parsing model")
    parser.parse(uff_model_path, network)

    print("building engine")
    with builder.build_cuda_engine(network) as engine:
        print("saving engine")
        with open(trt_engine_path, 'wb') as f:
            f.write(engine.serialize())

print("built .engine file successfully")
print("compiled TensorRT engine successfully")